In [1]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# model_name = "utrobinmv/t5_summary_en_ru_zh_base_2048"
model_name = "/root/lanyun-tmp/hf/t5_summary_en_ru_zh_base_2048"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name,
                                                   # load_in_8bit=True,
                                                   torch_dtype=torch.float16,
                                                   local_files_only=True).to(device)
model.eval()
print()

In [2]:
import re

text = "在北京冬奥会自由式滑雪女子坡面障碍技巧决赛中，中国选手谷爱凌夺得银牌。祝贺谷爱凌！今天上午，自由式滑雪女子坡面障碍技巧决赛举行。决赛分三轮进行，取选手最佳成绩排名决出奖牌。第一跳，中国选手谷爱凌获得69.90分。在12位选手中排名第三。完成动作后，谷爱凌又扮了个鬼脸，甚是可爱。第二轮中，谷爱凌在道具区第三个障碍处失误，落地时摔倒。获得16.98分。网友：摔倒了也没关系，继续加油！在第二跳失误摔倒的情况下，谷爱凌顶住压力，第三跳稳稳发挥，流畅落地！获得86.23分！此轮比赛，共12位选手参赛，谷爱凌第10位出场。网友：看比赛时我比谷爱凌紧张，加油！"

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

prefix = 'summary: '
src_text = prefix + text
input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids.to(device))

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_78182/2347900716.py:5: SyntaxWarning: invalid escape sequence '\s'
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))


['今天上午,自由式滑雪女子坡面障碍技巧决赛举行。决赛分三轮进行,取选手最佳成绩排名决出奖牌。第一跳,中国选手谷爱凌获得69.90分。']


In [3]:
integrated_save_path = "./results/summary_generator"
model.save_pretrained(integrated_save_path)
tokenizer.save_pretrained(integrated_save_path)

('./results/summary_generator/tokenizer_config.json',
 './results/summary_generator/special_tokens_map.json',
 './results/summary_generator/spiece.model',
 './results/summary_generator/added_tokens.json')